<a href="https://colab.research.google.com/github/Umesh9045/LP5/blob/main/Cuda_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-q3_qqszd
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-q3_qqszd
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-yqzdc65d/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
pip install nvcc4jupyter

In [4]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpcl34nxvn".


In [5]:
%%cuda

#include <stdio.h>

#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )

static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

const short N = 5 ;

__global__ void Vector_Addition ( const int *dev_a , const int *dev_b , int *dev_c)
{
      unsigned short tid = threadIdx.x ;

      if ( tid < N )
            dev_c [tid] = dev_a[tid] + dev_b[tid] ;

}


int main (void)
{
      int Host_a[N], Host_b[N], Host_c[N];


      int *dev_a , *dev_b, *dev_c ;


      HANDLE_ERROR ( cudaMalloc((void **)&dev_a , N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_b , N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_c , N*sizeof(int) ) );


      for ( int i = 0; i <N ; i++ )
      {
            Host_a[i] = i ;
            Host_b[i] = i*i ;
      }

      HANDLE_ERROR (cudaMemcpy (dev_a , Host_a , N*sizeof(int) , cudaMemcpyHostToDevice));
      HANDLE_ERROR (cudaMemcpy (dev_b , Host_b , N*sizeof(int) , cudaMemcpyHostToDevice));

      Vector_Addition <<< 1, N  >>> (dev_a , dev_b , dev_c ) ;

      HANDLE_ERROR (cudaMemcpy(Host_c , dev_c , N*sizeof(int) , cudaMemcpyDeviceToHost));

      for ( int i = 0; i<N; i++ )
                  printf ("%d + %d = %d\n", Host_a[i] , Host_b[i] , Host_c[i] ) ;

      cudaFree (dev_a) ;
      cudaFree (dev_b) ;
      cudaFree (dev_c) ;

      system("pause");
      return 0 ;

}

sh: 1: pause: not found
0 + 0 = 0
1 + 1 = 2
2 + 4 = 6
3 + 9 = 12
4 + 16 = 20





```
%%cuda

#include <stdio.h>

#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )
static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

# Size of arrays
const short N = 5 ;

# Function takes three parameters:
# dev_a: Pointer to the first input array.
# dev_b: Pointer to the second input array.
# dev_c: Pointer to the output array.
__global__ void Vector_Addition ( const int *dev_a , const int *dev_b , int *dev_c)
{
    #  calculates the thread index and stores in variable 'tid'
      unsigned short tid = threadIdx.x ;

      if ( tid < N )
            dev_c [tid] = dev_a[tid] + dev_b[tid] ;
}


int main (void)
{
      # Host vectors to store input vectors a, b, and output vector c
      int Host_a[N], Host_b[N], Host_c[N];

      # device pointer for device vectors
      int *dev_a , *dev_b, *dev_c ;

      #  Allocate memory on GPU dor device vector
      HANDLE_ERROR ( cudaMalloc((void **)&dev_a , N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_b , N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_c , N*sizeof(int) ) );

      # Initialize host arrays with values
      for ( int i = 0; i <N ; i++ )
      {
            Host_a[i] = i ;
            Host_b[i] = i*i ;
      }

      # Copy data from host to device
      HANDLE_ERROR (cudaMemcpy (dev_a , Host_a , N*sizeof(int) , cudaMemcpyHostToDevice));
      HANDLE_ERROR (cudaMemcpy (dev_b , Host_b , N*sizeof(int) , cudaMemcpyHostToDevice));

      # Launch kernel function 'vectorAdd' with one block containing 'N' threads.
      Vector_Addition <<< 1, N  >>> (dev_a , dev_b , dev_c ) ;

      # Copy result from device to host
      HANDLE_ERROR (cudaMemcpy(Host_c , dev_c , N*sizeof(int) , cudaMemcpyDeviceToHost));

      # Print result
      for ( int i = 0; i<N; i++ )
                  printf ("%d + %d = %d\n", Host_a[i] , Host_b[i] , Host_c[i] ) ;

      # Free device memory
      cudaFree (dev_a) ;
      cudaFree (dev_b) ;
      cudaFree (dev_c) ;

      system("pause");
      return 0 ;

}
```



In a **CUDA** program:
- The **main computer (CPU)** prepares the task and the data.
- It sends the data to the **graphics card (GPU)** for processing.
- The GPU does the heavy lifting, like adding big numbers together quickly.
- After finishing, the GPU sends the result back to the main computer.
- The main computer stores the result and can use it for further tasks.
